<a href="https://colab.research.google.com/github/GadielWillier/GadielWillier/blob/main/Webscrapping_Relat%C3%B3rios_Bacen_Recep%C3%A7%C3%A3o_Consentimento_Ativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Para ter acesso ao dados da página fazer esses passos:


1.   No Google Chrome clicar nos 3 pontinhos > Mais ferramentas > Ferramentas do desenvolvedor
2.   Ir na Aba Network e procura nos arquivos quais tem os dados a serem utilizados
3. Nas sub abas o Headers tem o request URL e no Payload tem o formato do json que deve ser construído





1: Geração das datas das requisições (Sextas-Feiras)

In [41]:
##Gerando as datas dos payloads das requisições
import datetime
import json

def obter_sextas_feiras_entre_datas(data_inicio, data_fim):
    sextas_feiras = []

    # Convertendo as strings de data para objetos datetime
    data_inicio = datetime.datetime.strptime(data_inicio, '%Y-%m-%d')
    data_fim = datetime.datetime.strptime(data_fim, '%Y-%m-%d')

    # Iterando entre as datas e adicionando as sextas-feiras à lista
    while data_inicio <= data_fim:
        if data_inicio.weekday() == 4: # 4 representa sexta-feira
            sextas_feiras.append(data_inicio.strftime("%Y-%m-%dT%H:%M:%S.%fZ"))

        # Incrementando um dia
        data_inicio += datetime.timedelta(days=1)

    return sextas_feiras

In [42]:
from datetime import date
# Coletando a data do dia corrente
hoje = date.today()

# Definindo as datas de início e fim
data_inicio = '2023-01-01'
data_fim = hoje.strftime('%Y-%m-%d')

#chamando a função para obter as sextas-feiras entre as datas
sextas = obter_sextas_feiras_entre_datas(data_inicio, data_fim)

2: Resultados Gerais

In [43]:
# Convertendo a lista para um dicionário com uma chave "dates" e a "role" da requisição, requisitos da API
payload_dict_geral = {"dates": sextas, "role":"client"}

# Convertendo o dicionário para uma JSON string e imprimindo
payload_json_geral = json.dumps(payload_dict_geral, indent=2)
print(payload_json_geral)

{
  "dates": [
    "2023-01-06T00:00:00.000000Z",
    "2023-01-13T00:00:00.000000Z",
    "2023-01-20T00:00:00.000000Z",
    "2023-01-27T00:00:00.000000Z",
    "2023-02-03T00:00:00.000000Z",
    "2023-02-10T00:00:00.000000Z",
    "2023-02-17T00:00:00.000000Z",
    "2023-02-24T00:00:00.000000Z",
    "2023-03-03T00:00:00.000000Z",
    "2023-03-10T00:00:00.000000Z",
    "2023-03-17T00:00:00.000000Z",
    "2023-03-24T00:00:00.000000Z",
    "2023-03-31T00:00:00.000000Z",
    "2023-04-07T00:00:00.000000Z",
    "2023-04-14T00:00:00.000000Z",
    "2023-04-21T00:00:00.000000Z",
    "2023-04-28T00:00:00.000000Z",
    "2023-05-05T00:00:00.000000Z",
    "2023-05-12T00:00:00.000000Z",
    "2023-05-19T00:00:00.000000Z",
    "2023-05-26T00:00:00.000000Z",
    "2023-06-02T00:00:00.000000Z",
    "2023-06-09T00:00:00.000000Z",
    "2023-06-16T00:00:00.000000Z",
    "2023-06-23T00:00:00.000000Z",
    "2023-06-30T00:00:00.000000Z",
    "2023-07-07T00:00:00.000000Z",
    "2023-07-14T00:00:00.000000Z",
    "

In [44]:
import requests
# Definindo a url do endpoint de consentimentos ativos
url_consents = "https://dashboard.openfinancebrasil.org.br/api/consents"

# Fazendo a chamada para a extração do report geral
response_general_report = requests.post(url_consents, data= payload_json_geral)

In [45]:
response_general_report.text

'[{"value":21206242,"date":"2023-01-06T00:00:00.000Z"},{"value":21910399,"date":"2023-01-13T00:00:00.000Z"},{"value":22254725,"date":"2023-01-20T00:00:00.000Z"},{"value":23359002,"date":"2023-01-27T00:00:00.000Z"},{"value":23879579,"date":"2023-02-03T00:00:00.000Z"},{"value":24518971,"date":"2023-02-10T00:00:00.000Z"},{"value":25085932,"date":"2023-02-17T00:00:00.000Z"},{"value":25526036,"date":"2023-02-24T00:00:00.000Z"},{"value":26218504,"date":"2023-03-03T00:00:00.000Z"},{"value":26766788,"date":"2023-03-10T00:00:00.000Z"},{"value":27172771,"date":"2023-03-17T00:00:00.000Z"},{"value":27785693,"date":"2023-03-24T00:00:00.000Z"},{"value":28314541,"date":"2023-03-31T00:00:00.000Z"},{"value":29500852,"date":"2023-04-07T00:00:00.000Z"},{"value":30202658,"date":"2023-04-14T00:00:00.000Z"},{"value":30643358,"date":"2023-04-21T00:00:00.000Z"},{"value":30643699,"date":"2023-04-28T00:00:00.000Z"},{"value":31516294,"date":"2023-05-05T00:00:00.000Z"},{"value":32083232,"date":"2023-05-12T00:00:0

In [46]:
!pip install pyspark
from pyspark.sql.functions import *

In [47]:
from pyspark.sql import SparkSession

# Inicia a SparkSession para instalar no google colab
spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

In [48]:
# Criação do dataframe de consentimentos ativos geral
df_general_report = (
    spark.read.json(spark.sparkContext.parallelize(json.loads(response_general_report.text),2))
    .withColumn("reference_week", to_date(col("date")))
    .drop("date")
    .withColumnRenamed("value", "active_consents")
    .select("reference_week", "active_consents")
)


In [49]:
df_general_report.show()

+--------------+---------------+
|reference_week|active_consents|
+--------------+---------------+
|    2023-01-06|       21206242|
|    2023-01-13|       21910399|
|    2023-01-20|       22254725|
|    2023-01-27|       23359002|
|    2023-02-03|       23879579|
|    2023-02-10|       24518971|
|    2023-02-17|       25085932|
|    2023-02-24|       25526036|
|    2023-03-03|       26218504|
|    2023-03-10|       26766788|
|    2023-03-17|       27172771|
|    2023-03-24|       27785693|
|    2023-03-31|       28314541|
|    2023-04-07|       29500852|
|    2023-04-14|       30202658|
|    2023-04-21|       30643358|
|    2023-04-28|       30643699|
|    2023-05-05|       31516294|
|    2023-05-12|       32083232|
|    2023-05-19|       32193166|
+--------------+---------------+
only showing top 20 rows



In [50]:
#df_general_report.write.saveAsTable('validation.consentimentos_ativos_dashboard_cidadao_geral', format= 'parquet', mode = 'overwrite')

3. Lista de Instituições e seus respectivos IDs de requisição

In [51]:
# Definindo a url do endpoint das organizações do open finance na lista
orgs_url = "https://dashboard.openfinancebrasil.org.br/api/organisations"

# Receptoras
receptoras_payload = {
    "role": "client",
    "phase": "transactional-data",
    "collection": "consents_weekly"
}

# Transmissoras
transmissoras_payload = {
    "role": "server",
    "phase": "transactional-data",
    "collection": "consents_weekly"
}

In [52]:
response_receptoras = requests.post(orgs_url, data= json.dumps(receptoras_payload)) #label: nome da IF, #value: id da requisição
response_receptoras_dict = json.loads(response_receptoras.text)

In [53]:
df_receptoras_total = (
    spark.read.json(spark.sparkContext.parallelize(response_receptoras_dict, 2))
    .select("label", "value")
    .withColumnRenamed("label", "institution")
    .withColumnRenamed("value", "institution_api_id")
)

In [54]:
#display(df_receptoras_total)
df_receptoras_total.show()

+--------------------+--------------------+
|         institution|  institution_api_id|
+--------------------+--------------------+
|           BANCO BMG|4ae2cefb-e3c1-562...|
|     BANCO BOCOM BBM|4a7250ec-eac5-5d8...|
|   BANCO BTG PACTUAL|0bd0b05e-ce18-510...|
|BANCO DE LA NACIO...|0e7f1342-d2bf-52f...|
|         BANCO DIGIO|27aea8f6-2119-55f...|
|     BANCO DO BRASIL|9415f224-9f58-56b...|
|   BANCO DO NORDESTE|a60b7fb5-f7f3-553...|
|      BANCO ORIGINAL|d3f8a4f7-7b6e-5b8...|
|           BANCO PAN|b6a214c7-6332-5a4...|
|      BANCO PAULISTA|99aff3b0-5617-5fa...|
|         BANCO SAFRA|709138dd-6e9d-5f9...|
|        BANCO SOFISA|bb3b0993-554d-5ea...|
|    BANCO VOTORANTIM|062c5edb-4e83-500...|
|            BANCO XP|79717d71-0cf4-58f...|
|            BANRISUL|2a01e202-e8f0-5f5...|
|            BELVO IP|5883e379-43b1-418...|
|               BNDES|7d1cf083-fae2-559...|
|            BRADESCO|a72a6d4f-79be-536...|
|CAIXA ECONOMICA F...|c160a6f5-e5df-506...|
|             CELCOIN|c55ceb2e-c

In [55]:
receptoras_list = df_receptoras_total.collect()
receptoras_total_dict = [row.asDict() for row in receptoras_list]

In [56]:
from pyspark.sql.types import StringType, IntegerType, StructType, StructField

In [57]:
# Schema dos dados de recepção agregado por IF e semana
schema_if_reception_weekly = StructType(
    [
        StructField("institution_name", StringType(), True),
        StructField("reference_week", StringType(), True),
        StructField("active_consents", IntegerType(), True)
    ]
)

# Criando um dataframe vazio
weekly_consents_reception_total_df = spark.createDataFrame(data=[], schema=schema_if_reception_weekly)

In [58]:
receptoras_total_dict

[{'institution': 'BANCO BMG',
  'institution_api_id': '4ae2cefb-e3c1-5624-9448-b3aa4777c539'},
 {'institution': 'BANCO BOCOM BBM',
  'institution_api_id': '4a7250ec-eac5-5d8f-b7eb-dc0e8e880203'},
 {'institution': 'BANCO BTG PACTUAL',
  'institution_api_id': '0bd0b05e-ce18-5105-bbe9-ddf16a79eff9'},
 {'institution': 'BANCO DE LA NACION ARGENTINA',
  'institution_api_id': '0e7f1342-d2bf-52f6-adbe-3f137aea9449'},
 {'institution': 'BANCO DIGIO',
  'institution_api_id': '27aea8f6-2119-55f8-9553-5ad4b08eeb17'},
 {'institution': 'BANCO DO BRASIL',
  'institution_api_id': '9415f224-9f58-56b5-af4c-085b4438e4eb'},
 {'institution': 'BANCO DO NORDESTE',
  'institution_api_id': 'a60b7fb5-f7f3-5536-a41f-1bff137f47d1'},
 {'institution': 'BANCO ORIGINAL',
  'institution_api_id': 'd3f8a4f7-7b6e-5b82-bdbe-bd90414650f0'},
 {'institution': 'BANCO PAN',
  'institution_api_id': 'b6a214c7-6332-5a41-8464-a281fb9a4ca0'},
 {'institution': 'BANCO PAULISTA',
  'institution_api_id': '99aff3b0-5617-5fad-bc6e-66e320a

Recepção: Consentimentos Ativos por IF - Geral

In [59]:
for row_dict in receptoras_total_dict:
  # Payload individual das instituições
  payload_recepcao_total = {
      "dates": sextas,
      "clients": [row_dict["institution_api_id"]],
      "role":"client",
  }
  payload_recepcao_total_formatted = json.dumps(payload_recepcao_total, indent=2)
  # Fazendo a requisição e transformando a resposta em dicionários
  response_recepcao_total = requests.request("POST", url_consents, data=payload_recepcao_total_formatted)
  if response_recepcao_total.status_code != 200:
    print(f"Chamada para f{row_dict['institution']} falhou com código {response_recepcao_total.status_code}")

  else:
    response_recepcao_total_dict = json.loads(response_recepcao_total.text)
    if response_recepcao_total_dict != []:
      for day in response_recepcao_total_dict:
        day["institution_name"] = row_dict["institution"]
      df_temp = spark.read.json(spark.sparkContext.parallelize(response_recepcao_total_dict, 2)).select(
          "institution_name","date","value"
      )
      weekly_consents_reception_total_df = weekly_consents_reception_total_df.union(df_temp)
    else:
      print(f"O JSON convertido da IF {row_dict['institution']} é nulo.")

O JSON convertido da IF BANCO DE LA NACION ARGENTINA é nulo.
O JSON convertido da IF BANCO PAULISTA é nulo.
O JSON convertido da IF BANCO SOFISA é nulo.
O JSON convertido da IF BELVO IP é nulo.
O JSON convertido da IF CELCOIN é nulo.
O JSON convertido da IF CIELO é nulo.
O JSON convertido da IF CRYSTAL BMC é nulo.
O JSON convertido da IF NEON PAGAMENTOS é nulo.
O JSON convertido da IF QI é nulo.
O JSON convertido da IF RECARGAPAY é nulo.
O JSON convertido da IF SICOOB UNIQUE BR é nulo.
O JSON convertido da IF UNICRED DO BRASIL é nulo.
O JSON convertido da IF UNIPRIME CENTRAL é nulo.


In [60]:
weekly_consents_reception_total_df = weekly_consents_reception_total_df.withColumn(
    "reference_week", to_date(col("reference_week"))
).orderBy("institution_name","reference_week")

In [61]:
#display(weekly_consents_reception_total_df)
weekly_consents_reception_total_df.show()

+----------------+--------------+---------------+
|institution_name|reference_week|active_consents|
+----------------+--------------+---------------+
|       BANCO BMG|    2023-01-06|          21990|
|       BANCO BMG|    2023-01-13|          22792|
|       BANCO BMG|    2023-01-20|          23653|
|       BANCO BMG|    2023-01-27|          24184|
|       BANCO BMG|    2023-02-03|          24777|
|       BANCO BMG|    2023-02-10|          25546|
|       BANCO BMG|    2023-02-17|          25858|
|       BANCO BMG|    2023-02-24|          26519|
|       BANCO BMG|    2023-03-03|          26618|
|       BANCO BMG|    2023-03-10|          27062|
|       BANCO BMG|    2023-03-17|          27235|
|       BANCO BMG|    2023-03-24|          27512|
|       BANCO BMG|    2023-03-31|          28176|
|       BANCO BMG|    2023-04-07|          28714|
|       BANCO BMG|    2023-04-14|          28947|
|       BANCO BMG|    2023-04-21|          29122|
|       BANCO BMG|    2023-04-28|          29508|


In [62]:
#weekly_consents_reception_total_df.write.saveAsTable('validation.consentimentos_ativos_dashboard_cidadao', format = 'parquet', mode = 'overwrite')

Resultados Agregados por IF

In [63]:
response_transmissoras = requests.post(orgs_url, data= json.dumps(transmissoras_payload)) #label: nome da IF, #value: id da requisição
response_transmissoras_dict = json.loads(response_transmissoras.text)

display( spark.read.json(spark.sparkContext.parallelize(response_transmissoras_dict,2)))

DataFrame[label: string, value: string]

In [64]:
#lista de IFs transmissoaras

response_transmissoras = requests.post(orgs_url, data= json.dumps(transmissoras_payload)) #label: nome da IF, #value: id da requisição
response_transmissoras_dict = json.loads(response_transmissoras.text)

df_transmissoras_total = (
    spark.read.json(spark.sparkContext.parallelize(response_transmissoras_dict, 2))
    .select("label", "value")
    .withColumnRenamed("label", "institution")
    .withColumnRenamed("value", "institution_api_id")
)

transmissoras_list = df_transmissoras_total.collect()
transmissoras_total_dict = [row.asDict() for row in transmissoras_list]

In [65]:
# Schema dos dados de recepção agregados por IF e semana
schema_if_reception_by_if_weekly = StructType(
    [
        StructField("institution_reception_name", StringType(), True),
        StructField("institution_transmition_name", StringType(), True),
        StructField("reference_week", StringType(), True),
        StructField("active_consents", IntegerType(), True)
    ]
)

# Criando um dataframe vazio
schema_consents_reception_by_if_df = spark.createDataFrame(data=[], schema=schema_if_reception_by_if_weekly)

Recepção: Consentimentos ativos por IF receptora com agregação

In [66]:
for row_dict_receptora in receptoras_total_dict:
  for row_dict_transmissora in transmissoras_total_dict:
    # Payload individual das instituições
    payload_recepcao_por_if = {
        "date": sextas,
        "clients": [row_dict["institution_api_id"]],
        "servers": [row_dict["institution_api_id"]],
        "role":"client",
    }
    payload_recepcao_por_if_formatted = json.dumps(payload_recepcao_por_if, indent=2)
  # Fazendo a requisição e transformando a resposta em dicionários
    response_recepcao_por_if = requests.request("POST", url_consents, data=payload_recepcao_por_if_formatted)

    if response_recepcao_por_if.status_code != 200:
      print(f"Chamada para f{row_dict['institution']} falhou com código {response_recepcao_por_if.status_code}")

    response_recepcao_por_if_dict = json.loads(response_recepcao_por_if.text)

    if response_recepcao_por_if_dict != []:
      for day in response_recepcao_por_if_dict:
        day["institution_reception_name"] = row_dict_receptora["institution"]
        day["institution_transmition_name"] = row_dict_transmissora["institution"]
      df_temp = spark.read.json(spark.sparkContext.parallelize(response_recepcao_por_if_dict, 2)).select(
          "institution_reception_name","institution_transmition_name","date","value"
      )
      weekly_consents_reception_by_if_df = weekly_consents_reception_by_if_df.union(df_temp)
    else:
      print(f"O JSON convertido da IF {row_dict_receptora['institution']} com transmissão da IF {row_dict_transmissora['institution']} é nulo.")

O JSON convertido da IF BANCO BMG com transmissão da IF BANCO BMG é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO BOCOM BBM é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO BTG PACTUAL é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO DIGIO é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO DO BRASIL é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO DO NORDESTE é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO ORIGINAL é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO PAN é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO PAULISTA é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO SAFRA é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO SOFISA é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANCO VOTORANTIM é nulo.
O JSON convertido da IF BANCO BMG com transmissão da IF BANC

In [67]:
weekly_consents_reception_by_if_df = weekly_consents_reception_by_if_df.withColumn(
    "reference_week", to_date(col("reference_week"))
).orderBy("institution_reception_name","reference_week")

NameError: name 'weekly_consents_reception_by_if_df' is not defined

In [ ]:
weekly_consents_reception_by_if_df.show()

In [ ]:
#weekly_consents_reception_by_if_df.write.saveAsTable('validation.consentimentos_ativos_recepcao_por_if_transmissora_dashboard_cidadao', format = 'parquet', mode = 'overwrite')